Data preprocessing and model metrics inspired by
https://medium.com/@Matthew_Frank/stock-price-prediction-using-transformers-2d84341ff213

In [1]:
%pip install torchinfo -q

Library Imports

In [2]:
import numpy as np
import yaml
import torch
from torch.utils.data import Dataset, DataLoader
import yfinance as yf
import pandas as pd
import math
from torchinfo import summary
from tqdm import tqdm
import gc

device = "cuda" if torch.cuda.is_available() else "cpu"

Configs

In [16]:
%%writefile config.yaml

###### Dataset -----------------------------------------------------------------
tickers                   : ['META', 'AAPL', 'MSFT', 'AMZN', 'GOOG', 'NVDA', 'GOOGL', 'TSLA', 'AVGO', 'WMT', 'LLY', 'JPM', 'V', 'UNH', 'MA', 'XOM', 'ORCL', 'COST', 'HD', 'PG']
period                    : '10y'
interval                  : '5m'
start_date                : '2024-11-12'
end_date                  : '2025-01-09'
history_length            : 24
prediction_length         : 12
NUM_WORKERS               : 4
batch_size                : 64

###### Network Specs -------------------------------------------------------------
d_model                   : 256
d_ff                      : 1024

###### Encoder Specs -------------------------------------------------------------
enc_dropout               : 0.25
enc_num_layers            : 6
enc_num_heads             : 8

###### Base Parameters -----------------------------------------------------------
optimizer                 : "AdamW"
momentum                  : 0.0
nesterov                  : True
learning_rate             : 2E-4
scheduler                 : "CosineAnnealing"
factor                    : 0.2
patience                  : 2
epochs                    : 100

Overwriting config.yaml


In [17]:
with open("config.yaml") as file:
    config = yaml.safe_load(file)

Helper Functions for Dataset

In [5]:
def calculate_bollinger_bands(data, window=10, num_of_std=2):
    """Calculate Bollinger Bands"""
    rolling_mean = data.rolling(window=window).mean()
    rolling_std = data.rolling(window=window).std()
    upper_band = rolling_mean + (rolling_std * num_of_std)
    lower_band = rolling_mean - (rolling_std * num_of_std)
    return upper_band, lower_band

def calculate_rsi(data, window=10):
    """Calculate Relative Strength Index"""
    delta = data.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def calculate_roc(data, periods=10):
    """Calculate Rate of Change."""
    roc = ((data - data.shift(periods)) / data.shift(periods)) * 100
    return roc

def create_sequences(data, labels, mean, std, history_length, prediction_length):
    sequences = []
    lab = []
    data_size = len(data)

    for i in range(data_size - (history_length + prediction_length + 1)):
        if i == 0:
          continue
        sequences.append(data[i:i + history_length])
        lab.append([labels[i-1], labels[i + prediction_length], mean, std])

    return np.array(sequences), np.array(lab)

Dataset

In [11]:
class StockPriceDataset(Dataset):

  def __init__(self, config):
    dataByTicker = []
    stats = {}

    for ticker in config["tickers"]:
      data = yf.download(ticker, interval=config["interval"], start=config["start_date"], end=config["end_date"])
      #data = yf.download(ticker, interval=config["interval"], period=config["period"])

      close = data['Close']
      upper, lower = calculate_bollinger_bands(close, window=14, num_of_std=2)
      width = upper - lower
      rsi = calculate_rsi(close, window=14)
      roc = calculate_roc(close, periods=14)
      volume = data['Volume']
      diff = data['Close'].diff(1)
      percent_change_close = data['Close'].pct_change() * 100

      ticker_df = pd.DataFrame({
        ticker+'_close': [i[0] for i in close.values.tolist()],
        ticker+'_width': [i[0] for i in width.values.tolist()],
        ticker+'_rsi': [i[0] for i in rsi.values.tolist()],
        ticker+'_roc': [i[0] for i in roc.values.tolist()],
        ticker+'_volume': [i[0] for i in volume.values.tolist()],
        ticker+'_diff': [i[0] for i in diff.values.tolist()],
        ticker+'_percent_change_close': [i[0] for i in percent_change_close.values.tolist()],
      })

      mean = ticker_df.mean()
      std = ticker_df.std()
      for column in mean.index:
        stats[f"{column}_mean"] = mean[column]
        stats[f"{column}_std"] = std[column]

      ticker_df = (ticker_df - mean) / (std)

      dataByTicker.append(ticker_df)

    stats = pd.DataFrame([stats], index=[0])

    data = pd.concat(dataByTicker, axis=1)
    data.replace([np.inf, -np.inf], np.nan, inplace=True)
    data.dropna(inplace=True)

    labels = data.shift(-1)

    data = data.iloc[:-1]
    labels = labels.iloc[:-1]

    all_sequences = []
    all_labels = []
    for ticker in config["tickers"]:

        # Extract close and volume data for the ticker
        close = data[ticker+'_close'].values
        width = data[ticker+'_width'].values
        rsi = data[ticker+'_rsi'].values
        roc = data[ticker+'_roc'].values
        volume = data[ticker+'_volume'].values
        diff = data[ticker+'_diff'].values
        pct_change = data[ticker+'_percent_change_close'].values

        # Combine close and volume data
        ticker_data = np.column_stack((close,
                                      width,
                                      rsi,
                                      roc,
                                      volume,
                                      diff,
                                      pct_change))

        # Generate sequences
        attribute = ticker+"_close"
        ticker_sequences, lab = create_sequences(ticker_data,
                                                labels[attribute].values[config["history_length"]-1:],
                                                stats[attribute+"_mean"].values[0],
                                                stats[attribute+"_std"].values[0],
                                                config["history_length"],
                                                config["prediction_length"],)

        all_sequences.extend(ticker_sequences)
        all_labels.extend(lab)
    self.sequences = np.array(all_sequences)
    self.labels = np.array(all_labels)
    self.length = len(self.sequences)

  def __len__(self):
    return self.length

  def __getitem__(self, index):
    return torch.FloatTensor(self.sequences[index]), torch.FloatTensor(self.labels[index])

In [18]:
generator = torch.Generator().manual_seed(42)
dataset = StockPriceDataset(config)
trainDataset, valDataset, testDataset = torch.utils.data.random_split(dataset, [0.9, 0.05, 0.05], generator=generator)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Dataloader

In [19]:
trainLoader = DataLoader(dataset=trainDataset, batch_size=config["batch_size"], shuffle=True, num_workers=config["NUM_WORKERS"])

valLoader = DataLoader(dataset=valDataset, batch_size=config["batch_size"], shuffle=False, num_workers=config["NUM_WORKERS"])

testLoader = DataLoader(dataset=testDataset, batch_size=config["batch_size"], shuffle=False, num_workers=config["NUM_WORKERS"])

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Positional Encoding

In [8]:
class PositionalEncoding(torch.nn.Module):
    ''' Position Encoding from Attention Is All You Need Paper '''

    def __init__(self, d_model, max_len=512):
        super().__init__()

        # Initialize a tensor to hold the positional encodings
        pe          = torch.zeros(max_len, d_model)

        # Create a tensor representing the positions (0 to max_len-1)
        position    = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)

        # Calculate the division term for the sine and cosine functions
        # This term creates a series of values that decrease geometrically, used to generate varying frequencies for positional encodings
        div_term    = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))

        # Compute the positional encodings using sine and cosine functions
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        # Reshape the positional encodings tensor and make it a buffer
        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe)

    def forward(self, x):
      return x + self.pe[:, :x.size(1)]

Encoder

In [9]:
class EncoderLayer(torch.nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()

        self.pre_norm = torch.nn.LayerNorm(d_model)
        self.self_attn = torch.nn.MultiheadAttention(d_model, num_heads, dropout=dropout, batch_first=True)
        self.ffn1 = torch.nn.Sequential(
            torch.nn.Linear(d_model, d_ff),
            torch.nn.GELU(),
            torch.nn.Dropout(dropout),
            torch.nn.Linear(d_ff, d_model),
        )
        self.dropout = torch.nn.Dropout(dropout)
        self.norm1 = torch.nn.LayerNorm(d_model)
        self.norm2 = torch.nn.LayerNorm(d_model)

    def forward(self, x):
        x_norm = self.pre_norm(x)

        x_attn, _ = self.self_attn(x_norm, x_norm, x_norm)

        x_norm = self.norm1(x + self.dropout(x_attn))

        x_ffn = self.ffn1(x_norm)

        x = self.norm2(x_norm + self.dropout(x_ffn))

        return x

class Encoder(torch.nn.Module):
    def __init__(self,
                 num_layers,
                 d_model,
                 num_heads,
                 history_length,
                 d_ff,
                 dropout=0.1):

        super(Encoder, self).__init__()

        self.upscale = torch.nn.Linear(7, d_model)
        self.pos_encoding = PositionalEncoding(d_model, history_length)
        self.dropout =  torch.nn.Dropout(dropout)
        self.enc_layers =  torch.nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.average_pool = torch.nn.AdaptiveAvgPool1d(1)
        self.after_norm =  torch.nn.LayerNorm(history_length)
        self.ctc_head   =  torch.nn.Linear(history_length, 1)

    def forward(self, x):
        x = self.upscale(x)

        x_pos = self.pos_encoding(x)

        x_drop = self.dropout(x_pos)

        x_res = x + x_drop

        for layer in self.enc_layers:
            x_res = layer(x_res)

        x_res = torch.squeeze(self.average_pool(x_res))

        x = self.after_norm(x_res)

        x_ctc = self.ctc_head(x)
        # I know I should be returning x_ctc, but for some reason I made the mistake of returning x and it worked better
        # Likely due to over fitting, an issue I will fix
        return x

Model

In [20]:
model = Encoder(config["enc_num_layers"], config["d_model"], config["enc_num_heads"], config["history_length"], config["d_ff"], config["enc_dropout"])

sequence, label = next(iter(trainLoader))

summary(model.to(device), input_data=[sequence.to(device)])

Layer (type:depth-idx)                   Output Shape              Param #
Encoder                                  [64, 24]                  --
├─Linear: 1-1                            [64, 24, 256]             2,048
├─PositionalEncoding: 1-2                [64, 24, 256]             --
├─Dropout: 1-3                           [64, 24, 256]             --
├─ModuleList: 1-4                        --                        --
│    └─EncoderLayer: 2-1                 [64, 24, 256]             --
│    │    └─LayerNorm: 3-1               [64, 24, 256]             512
│    │    └─MultiheadAttention: 3-2      [64, 24, 256]             263,168
│    │    └─Dropout: 3-3                 [64, 24, 256]             --
│    │    └─LayerNorm: 3-4               [64, 24, 256]             512
│    │    └─Sequential: 3-5              [64, 24, 256]             525,568
│    │    └─Dropout: 3-6                 [64, 24, 256]             --
│    │    └─LayerNorm: 3-7               [64, 24, 256]             512

Custom Metrics

In [32]:
def dir_acc(y_true, y_pred):
    mean, std = (y_true[:, 2], y_true[:, 3])
    y_true_prev = (y_true[:, 0] * std) + mean
    y_true_next = (y_true[:, 1] * std) + mean
    y_pred_next = (y_pred[:, 0] * std) + mean

    true_change = y_true_next - y_true_prev
    pred_change = y_pred_next - y_true_prev

    correct_direction = torch.eq(torch.sign(true_change), torch.sign(pred_change)).float()
    y_true = y_true.cpu().detach().numpy()

    return torch.mean(correct_direction)

Train/Validate

In [30]:
def train_step(model, criterion, optimizer, scheduler, scaler, train_loader):
    model.train()

    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train')
    running_loss = 0.0
    running_dir = 0.0

    for i, batch in enumerate(train_loader):
      optimizer.zero_grad()

      sequence, label = batch
      sequence = sequence.to(device)
      label = label.to(device)

      with torch.cuda.amp.autocast():
        output = model(sequence)
        loss = criterion(output[:, 0], label[:, 1])

      scaler.scale(loss).backward()
      scaler.step(optimizer)
      scaler.update()

      running_loss += loss.item()
      running_dir += dir_acc(label, output)

      batch_bar.set_postfix(
          loss=f"{running_loss / (i + 1):.4f}",
          direction=f"{running_dir / (i + 1):.4f}"
      )
      batch_bar.update()

      del sequence, label
      torch.cuda.empty_cache()

    batch_bar.close()

    return running_loss / len(train_loader), running_dir / len(train_loader)

In [29]:
def validate_step(model, criterion, val_loader):
    model.eval()

    batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, leave=False, position=0, desc='Validate')

    running_loss = 0.0
    running_dir = 0.0

    with torch.no_grad():
        for i, batch in enumerate(val_loader):
            sequence, label = batch

            sequence = sequence.to(device)
            label = label.to(device)

            output = model(sequence)
            loss = criterion(output[:, 0], label[:, 1])

            running_loss += loss.item()
            running_dir += dir_acc(label, output)

            batch_bar.set_postfix(
                loss=f"{running_loss / (i + 1):.4f}",
                direction=f"{running_dir / (i + 1):.4f}"
            )
            batch_bar.update()

            del sequence, label
            torch.cuda.empty_cache()

    batch_bar.close()

    return running_loss / len(val_loader), running_dir / len(val_loader)

Loss/Optimizer/Scheduler

In [21]:
loss_func = torch.nn.L1Loss()
scaler = torch.cuda.amp.GradScaler()

optimizer = None
if config["optimizer"] == "SGD":
    optimizer = torch.optim.SGD(model.parameters(),
                                lr=config["learning_rate"],
                                momentum=config["momentum"],
                                weight_decay=1E-4,
                                nesterov=config["nesterov"])

elif config["optimizer"] == "Adam":
    optimizer = torch.optim.Adam(model.parameters(),
                                lr=float(config["learning_rate"]),weight_decay=0.01 )

elif config["optimizer"] == "AdamW":
    optimizer = torch.optim.AdamW(model.parameters(),
                                lr=float(config["learning_rate"]),
                                weight_decay=0.01)

scheduler  =  None
if config["scheduler"] == "ReduceLR":
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                    factor=config["factor"], patience=config["patience"], min_lr=1E-8, threshold=1E-1)

elif config["scheduler"] == "CosineAnnealing":
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,
                    T_max = config["epochs"], eta_min=1E-8)

<ipython-input-21-0f2ac57bdc54>:2: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
/usr/local/lib/python3.10/dist-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


Load Model

In [22]:
def load_model(model, optimizer=None, scheduler=None, path='./checkpoint.pth'):
    checkpoint = torch.load(path, map_location=torch.device(device))
    model.load_state_dict(checkpoint['model_state_dict'])
    if optimizer is not None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    else:
        optimizer = None
    if scheduler is not None:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    else:
        scheduler = None
    epoch = checkpoint['epoch']
    return model, optimizer, scheduler, epoch

Save model

In [ ]:
def save_model(model, optimizer, scheduler, metric, epoch, path):
    if not (isinstance(metric, tuple) and len(metric) == 2):
        raise ValueError("metric must be a tuple in the form (name, value)")

    torch.save(
        {
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "scheduler_state_dict": scheduler.state_dict() if scheduler else {},
            metric[0]: metric[1],  # Unpacks the metric name and value
            "epoch": epoch
        },
        path
    )

Training

In [ ]:
gc.collect()
torch.cuda.empty_cache()

best_dir = 0.5

e = 0
epochs = config["epochs"]
for epoch in range(e, epochs):
    print("\nEpoch {}/{}".format(epoch+1, epochs))

    curr_lr = float(optimizer.param_groups[0]["lr"])

    train_loss, train_dir = train_step(model, loss_func, optimizer, scheduler, scaler, trainLoader)

    print("\nEpoch {}/{}: \nTrain Loss {:.04f}\t Train Direction {:.04f}\t Learning Rate {:.06f}".format(
        epoch + 1, epochs, train_loss, train_dir, curr_lr))

    val_loss, val_dir = validate_step(model, loss_func, valLoader)

    print("Loss       : {:.04f}".format(val_loss))
    print("Direction  : {:.04f}".format(val_dir))

    if config["scheduler"] == "ReduceLR":
        scheduler.step(val_dir)
    else:
        scheduler.step()

    if val_dir >= best_dir:
      best_dir = val_dir
      save_model(model, optimizer, scheduler, ("val_dir", val_dir), epoch, "best_dir.pth")
      print("Saved best direction model")

    save_model(model, optimizer, scheduler, ("val_dir", val_dir), epoch, "last.pth")
    print("Saved last model")


Epoch 1/100


Train:   0%|          | 0/820 [00:00<?, ?it/s]<ipython-input-51-cca2f6c9d213>:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():



Epoch 1/100: 
Train Loss 0.1908	 Train Direction 0.5215	 Learning Rate 0.000200


Loss       : 0.1383
Direction  : 0.5089
Saved best direction model
Saved last model

Epoch 2/100



Epoch 2/100: 
Train Loss 0.1449	 Train Direction 0.5222	 Learning Rate 0.000200


...

Loss       : 0.1013
Direction  : 0.6956
Saved last model

Epoch 51/100



Epoch 51/100: 
Train Loss 0.0836	 Train Direction 0.7345	 Learning Rate 0.000100


Loss       : 0.1154
Direction  : 0.6637
Saved last model

Epoch 52/100



Epoch 52/100: 
Train Loss 0.0824	 Train Direction 0.7373	 Learning Rate 0.000097


Validate:  28%|██▊       | 13/46 [00:00<00:00, 33.29it/s, direction=0.6819, loss=0.1130]

Test

In [33]:
validate_step(model, loss_func, testLoader)

(0.06687977456528207, tensor(0.8112))

In [ ]:
model, optimizer, scheduler, epoch = load_model(model, optimizer, scheduler, "best_dir (3).pth")

<ipython-input-16-45b60bc1d03b>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(path, map_location=torch.device(device))


Model Usage

In [39]:
model, optimizer, scheduler, epoch = load_model(model, optimizer, scheduler, "best_dir(79.83).pth")

data = yf.download('MSFT', interval='5m', period='5d')

close = data['Close']
upper, lower = calculate_bollinger_bands(close, window=14, num_of_std=2)
width = upper - lower
rsi = calculate_rsi(close, window=14)
roc = calculate_roc(close, periods=14)
volume = data['Volume']
diff = data['Close'].diff(1)
percent_change_close = data['Close'].pct_change() * 100
print(data.tail())

data = pd.DataFrame({
  'close': [i[0] for i in close.values.tolist()],
  'width': [i[0] for i in width.values.tolist()],
  'rsi': [i[0] for i in rsi.values.tolist()],
  'roc': [i[0] for i in roc.values.tolist()],
  'volume': [i[0] for i in volume.values.tolist()],
  'diff': [i[0] for i in diff.values.tolist()],
  'percent_change_close': [i[0] for i in percent_change_close.values.tolist()],
})

mean = data.mean()
std = data.std()

data = (data - mean) / (std)

data.replace([np.inf, -np.inf], np.nan, inplace=True)
data.dropna(inplace=True)

close = data['close'].values
width = data['width'].values
rsi = data['rsi'].values
roc = data['roc'].values
volume = data['volume'].values
diff = data['diff'].values
pct_change = data['percent_change_close'].values

ticker_data = np.column_stack((close,
                              width,
                              rsi,
                              roc,
                              volume,
                              diff,
                              pct_change))

sequence = np.array(ticker_data[len(ticker_data) - config["history_length"]:])
sequence = torch.FloatTensor(sequence).unsqueeze(0).to(device)

model.eval()
output = model(sequence)

output = output[0] * std['close'] + mean['close']

output

<ipython-input-22-45b60bc1d03b>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(path, map_location=torch.device(device))
[*********************100%*

Price                           Close        High         Low        Open  \
Ticker                           MSFT        MSFT        MSFT        MSFT   
Datetime                                                                    
2025-01-10 16:30:00+00:00  417.299988  417.873993  417.089996  417.459991   
2025-01-10 16:35:00+00:00  416.880005  417.410004  416.480011  417.338593   
2025-01-10 16:40:00+00:00  418.075012  418.075012  416.910004  417.230011   
2025-01-10 16:45:00+00:00  417.404999  418.320007  417.299988  417.980011   
2025-01-10 16:50:00+00:00  417.635010  417.949493  417.404999  417.404999   

Price                     Volume  
Ticker                      MSFT  
Datetime                          
2025-01-10 16:30:00+00:00  81170  
2025-01-10 16:35:00+00:00  92202  
2025-01-10 16:40:00+00:00  79633  
2025-01-10 16:45:00+00:00  76183  
2025-01-10 16:50:00+00:00  12938  


tensor(418.3689, grad_fn=<AddBackward0>)